# Practice Project 2.1 - Preparing School Data

## Business Understanding
A school district wants to predict the per pupil costs of a school based on some high level summary data about the school. This way they’ll have a good estimation of how well a school is managing its costs relative to what the model would predict. You’ve been asked to to prepare the data for modelling.

## Data Understanding
You’ve been given four CSV files that contain data for two different school districts. You can find these files at the bottom of the page.

- DistrictA_Attendance - This file contains average daily attendance, percent attendance, and pupil-teacher ratio data for the 25 schools in district A.

- DistrictA_Finance - This file contains average monthly teacher salary and per pupil cost data for the 25 schools in district A.

- DistrictB_Attendance - This file contains average daily attendance, percent attendance, and pupil-teacher ratio data for the 21 schools in district B.

- DistrictB_Finance - This file contains average monthly teacher salary and per pupil cost data for the 21 schools in district B.

In [13]:
# load module
import pandas as pd

In [14]:
# load attendance and finance data
a_attendance = pd.read_csv('districta-attendance.csv')
b_attendance = pd.read_csv('districtb-attendance.csv')
a_finance = pd.read_csv('districta-finance.csv')
b_finance = pd.read_csv('districtb-finance.csv')

# clean up data
attendance_columns = ['School', 'Average daily Attendance', 'Percent Attendance', 'Pupil/Teacher ratio']
a_attendance = a_attendance[attendance_columns]

# Join data
a_b_attendance = pd.concat([a_attendance, b_attendance])
a_b_finance = pd.concat([a_finance, b_finance])
